# 제주 데이터허브 크롤링 코드

* 작성자: 송채은(2021), 김현지(2022-04)

---
* 제주 데이터허브는 원본 데이터가 전송되는 url을 추적하여 가져오는 방식으로 크롤링합니다. 추출한 데이터(stock_data)를 바탕으로 제주 데이터허브에서 제공하는 메타데이터와 id를 추출하고, 만약 stock 데이터에 해당 항목이 없는 경우 만들어야 합니다. 따라서 크롤링 시 개별 데이터세트의 메타데이터 항목을 잘 파악하고 코드를 수정해주세요.





In [4]:
from urllib.request import urlopen
from urllib import request
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import re
import json
import random
from glob import glob
from tqdm.notebook import tqdm

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


제주 데이터허브의 메타데이터 확인 후 수정 필요함

In [8]:
import xmltodict
import json
from pandas import json_normalize
import xml.dom.minidom

In [9]:
# 코드 돌린 날짜
import datetime
today = datetime.date.today().strftime('%Y-%m-%d')
print(today)

2022-07-17


In [10]:
#custom_header을 통해 아닌 것 처럼 위장하기
#해당 접속 사이트가 아닌 원본데이터가 오는 url 추적. network에서 가지고 온다.
custom_header = {
    'Accept': 'application/json, text/plain, */*', #이거 때문인가?
    'Referer': 'https://www.jejudatahub.net/data/list',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36' #이거때문인가?
     }

# 2022-03 : 수량 변동으로 url length 변수 수정
url = "https://www.jejudatahub.net/api/data?includedKeywords=&excludedKeywords=&categories=&userId=&dataType=&orderBy=createAt&authYn=false&keyword=&isPaging=true&start=0&pageNumber=1&length=1200"

req = requests.get(url, headers = custom_header)    #custom_header를 사용하지 않으면 접근 불가

if req.status_code == requests.codes.ok:    
    print("접속 성공")
    stock_data = json.loads(req.text)        #json에 반환된 데이터가 들어가 있다.

else:
    print("Error code")

접속 성공


In [11]:
total = stock_data['data']
df = json_normalize(stock_data['data'])
df.columns

Index(['id', 'dataKey', 'category', 'categoryName', 'title', 'description',
       'startDate', 'endDate', 'autoUpdateYn', 'autoUpdatePeriod', 'authYn',
       'viewCount', 'createAt', 'createBy', 'createByName', 'creator', 'owner',
       'updateAt', 'deleteAt', 'isNewData', 'isFavorite', 'totalFavCount',
       'totalProjCount', 'dataApiCount', 'dataFileCount', 'dataLinkCount',
       'authYnCount', 'dataApi', 'dataLink', 'infographicGeneral',
       'infographicMultimedia', 'projects', 'readyStateCount',
       'totalQuotaRequest', 'totalErrorReportCount', 'csvData', 'dataFile.id',
       'dataFile.createAt', 'dataFile.createBy', 'dataFile.fileInfos',
       'file.id', 'file.createAt', 'file.createBy', 'file.fileInfos',
       'dataApi.id', 'dataApi.url', 'dataApi.dataFormat', 'dataApi.methodType',
       'dataApi.characterSet', 'dataApi.quota', 'dataApi.file.id',
       'dataApi.file.createAt', 'dataApi.file.createBy',
       'dataApi.file.fileInfos', 'dataApi.dataApiElements', 'da

In [12]:
# 필요 없는 항목 제거
df.drop(['totalProjCount', 'dataApiCount','dataFileCount', 'dataLinkCount', 'authYnCount', 'dataApi', 'dataLink', 
        'projects', 'readyStateCount', 'totalQuotaRequest', 'totalErrorReportCount', 'csvData', 'dataFile'], axis = 1, inplace =True)

df.drop(['autoUpdateYn', 'autoUpdatePeriod', 'authYn', 'createBy', 'creator', 'deleteAt',
        'isNewData', 'isFavorite', 'dataFile.createAt', 'dataFile.createBy', 'file.createAt', 'file.createBy',
        'dataApi.file.createAt', 'dataApi.file.createBy'], axis = 1, inplace =True)

df.drop('file.fileInfos', axis = 1, inplace = True)

In [13]:
# infographicGeneral
df['infographicGeneral'] = df['infographicGeneral'].apply(lambda a : None if a == [] else a)

# infographicMultimedia
df['infographicMultimedia'] = df['infographicMultimedia'].apply(lambda a : None if a == [] else a)

# accessUrl 만들기
df['accessUrl'] = df['id'].apply(lambda s : 'https://www.jejudatahub.net/data/view/data/'+ str(s) )


In [14]:
import numpy as np
df = df.replace({np.NaN : None})

In [15]:
# service Type 만들기
serviceTypeList = []
for idx in df.index : 
    serviceType = []
    if df.loc[idx, 'dataFile.fileInfos'] == None :
        pass
    else :
        serviceType.append('FILE')
    
    if df.loc[idx, 'dataApi.id'] == None :
        pass
    else :
        serviceType.append('API')
        
    if df.loc[idx, 'infographicGeneral'] == None :
        pass
    else :
        serviceType.append('차트')
    
    if df.loc[idx, 'infographicMultimedia'] == None :
        pass
    else :
        serviceType.append('영상')
        
    if df.loc[idx, 'dataLink.url'] == None :
        pass
    else :
        serviceType.append('LINK')    
    
    serviceTypeList.append(serviceType)

In [16]:
df['serviceType'] = serviceTypeList
df['serviceType'] = df['serviceType'].apply(lambda a : None if a == [] else a)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       1176 non-null   int64 
 1   dataKey                  1176 non-null   object
 2   category                 1176 non-null   object
 3   categoryName             1176 non-null   object
 4   title                    1176 non-null   object
 5   description              1176 non-null   object
 6   startDate                1176 non-null   object
 7   endDate                  664 non-null    object
 8   viewCount                1176 non-null   int64 
 9   createAt                 1176 non-null   object
 10  createByName             1176 non-null   object
 11  owner                    1176 non-null   object
 12  updateAt                 1176 non-null   object
 13  totalFavCount            1176 non-null   int64 
 14  infographicGeneral       58 non-null    

In [18]:
df_file = df[['id', 'dataFile.id', 'dataFile.fileInfos']]
df_file = df_file.dropna(subset = ['dataFile.fileInfos'])
df_file.reset_index(drop = True, inplace =True)

In [19]:
df_file

,id,dataFile.id,dataFile.fileInfos
0,1218,3125.0,"[{'id': 3697, 'fileId': 3125, 'fileName': '제주도..."
1,1216,3106.0,"[{'id': 3667, 'fileId': 3106, 'fileName': '체류빈..."
2,1215,3104.0,"[{'id': 3660, 'fileId': 3104, 'fileName': '체류빈..."
3,1214,3102.0,"[{'id': 3653, 'fileId': 3102, 'fileName': '체류빈..."
4,1213,3100.0,"[{'id': 3646, 'fileId': 3100, 'fileName': '체류빈..."
...,...,...,...
588,189,1380.0,"[{'id': 1558, 'fileId': 1380, 'fileName': '내국인..."
589,188,193.0,"[{'id': 193, 'fileId': 193, 'fileName': '제주공공와..."
590,187,819.0,"[{'id': 977, 'fileId': 819, 'fileName': '전국코로나..."
591,743,1302.0,"[{'id': 1451, 'fileId': 1302, 'fileName': 'BUS..."


In [20]:
file_data = df_file.loc[0, 'dataFile.fileInfos']
file_data

[{'id': 3697,
  'fileId': 3125,
  'fileName': '제주도 다문화 가정을 위한 가정통신문 단어 쉬운 한국말 풀이.csv',
  'fileSize': 90672,
  'contentType': 'text/csv',
  'saveFileName': '2,06590f5792be79',
  'savePath': None,
  'downloadCount': 20,
  'createAt': '2022-04-30T16:03:52',
  'createBy': 68}]

In [21]:
temp = json_normalize(file_data)
temp['catalog_id'] = df_file.loc[0, 'id']
temp['catalog_dataFile.id'] = df_file.loc[0, 'dataFile.id']

In [22]:
file_data

[{'id': 3697,
  'fileId': 3125,
  'fileName': '제주도 다문화 가정을 위한 가정통신문 단어 쉬운 한국말 풀이.csv',
  'fileSize': 90672,
  'contentType': 'text/csv',
  'saveFileName': '2,06590f5792be79',
  'savePath': None,
  'downloadCount': 20,
  'createAt': '2022-04-30T16:03:52',
  'createBy': 68}]

In [23]:
file_result = pd.DataFrame()

for idx in df_file.index : 
    file_data = df_file.loc[idx, 'dataFile.fileInfos']
    
    for i in file_data :
        temp = json_normalize(i)
        temp['catalog_id'] = df_file.loc[idx, 'id']
        temp['catalog_dataFile.id'] = df_file.loc[idx, 'dataFile.id']
        
        file_result = file_result.append(temp)
        
file_result.reset_index(drop = True, inplace = True)

C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1679377313.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_result = file_result.append(temp)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1679377313.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_result = file_result.append(temp)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1679377313.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_result = file_result.append(temp)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1679377313.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_result = file_result.append(temp)
C:\Users\sangw\AppData\Local\Tem

In [24]:
file_result['downloadUrl'] = file_result.apply(lambda a :
                                               'https://www.jejudatahub.net/api/file?id={0}&fileId={1}&fileName={2}&fileSize={3}&contentType={4}&saveFileName={5}&downloadCount={6}&createAt={7}&createBy={8}'.format(
                                               a['id'], a['fileId'], a['fileName'], a['fileSize'], a['contentType'], a['saveFileName'], a['downloadCount'], a['createAt'], a['createBy']), axis = 1)
file_result.drop('savePath', axis = 1, inplace =True )

In [40]:
driver = webdriver.Chrome()

In [ ]:
for i in range(1,len(file_result)):
    driver.get(str(file_result.loc[i,'downloadUrl']))
    driver.implicitly_wait(3)

In [32]:
file_result = file_result[['catalog_id',
       'catalog_dataFile.id','id', 'fileId', 'fileName', 'fileSize', 'contentType', 'saveFileName',
       'downloadCount', 'createAt', 'createBy', 'downloadUrl']]

In [40]:
file_result.to_csv('../result/제주데이터허브-파일-2022-04-16.csv', index = False, encoding = 'utf-8')

In [33]:
# api


df_api = df[['id', 'dataApi.id', 'dataApi.url', 'dataApi.dataFormat', 'dataApi.methodType',
       'dataApi.characterSet', 'dataApi.quota', 'dataApi.file.id',
       'dataApi.file.fileInfos', 'dataApi.dataApiElements']]
df_api = df_api.dropna(subset = ['dataApi.id'])
df_api.reset_index(inplace =True, drop = True)

In [34]:
# 기술문서 다운로드 url 만들기

df_api_temp = pd.DataFrame()

for idx in df_api.index : 
    file_api = df_api.loc[idx, 'dataApi.file.fileInfos']
    df_api_temp = df_api_temp.append(file_api)

df_api_temp['document'] = df_api_temp.apply(lambda a : 'https://www.jejudatahub.net/api/file?id={0}&fileId={1}&fileName={2}&fileSize={3}&contentType={4}&saveFileName={5}&downloadCount={6}&createAt={7}&createBy={8}'.format(a.id, a.fileId, a.fileSize, a.contentType, a.saveFileName, a.savePath, a.downloadCount, a.createAt, a.createBy), axis = 1)
df_api_temp.reset_index(inplace =True, drop = True)
df_api['documnet'] = df_api_temp['document']

C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1686290705.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_api_temp = df_api_temp.append(file_api)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1686290705.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_api_temp = df_api_temp.append(file_api)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1686290705.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_api_temp = df_api_temp.append(file_api)
C:\Users\sangw\AppData\Local\Temp\ipykernel_10004\1686290705.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_api_temp = df_api_temp.append(file_api)
C:\Users\sangw\AppDa

In [35]:
# 컬럼 정리
df_api.rename(columns = {'id' : 'catalog_id'}, inplace = True)
df_api.drop(['dataApi.file.fileInfos', 'dataApi.dataApiElements', 'dataApi.file.id'], axis = 1, inplace = True)

In [49]:
df_api.loc[27, 'documnet']

'https://www.jejudatahub.net/api/file?id=2497&fileId=2139&fileName=70656&fileSize=application/haansofthwp&contentType=1,02e9da80c14488&saveFileName=None&downloadCount=55&createAt=2021-10-20T19:33:05&createBy=1'

In [41]:
for i in range(1,len(df_api)):
    driver.get(str(df_api.loc[i,'documnet']))
    driver.implicitly_wait(3)

In [202]:
df_api.to_csv('../result/제주데이터허브-api-2022-04-12.csv', index = False, encoding = 'utf-8')

In [50]:
# link

df_link = df[['id', 'dataLink.id', 'dataLink.url', 'infographicMultimedia']]
df_link = df_link.dropna(subset = ['dataLink.id', 'infographicMultimedia'], how = 'all')
df_link.reset_index(inplace =True, drop = True)
df_link.rename(columns = {'id' : 'catalog_id'}, inplace = True)

In [51]:
df_link['MultimediaUrl'] = df_link['infographicMultimedia'].apply(lambda a : a if a == None else a[0]['url'])
df_link.drop('infographicMultimedia', inplace =True, axis = 1)

In [203]:
df_link.to_csv('../result/제주데이터허브-link-multiMedia-2022-04-12.csv', index = False, encoding = 'utf-8')

In [52]:
df_link

,catalog_id,dataLink.id,dataLink.url,MultimediaUrl
0,1049,None,None,https://www.youtube.com/embed/tD2kmqNLHP0
1,1048,None,None,https://www.youtube.com/embed/GgPis2dwPXQ
2,1047,None,None,https://www.youtube.com/embed/xUcaimWO410
3,868,None,None,https://www.youtube.com/embed/5LI2ok_oaA8
4,867,None,None,https://www.youtube.com/embed/rcxOZy81S-w
...,...,...,...,...
530,9,8.0,https://www.data.go.kr/data/15056048/fileData.do,None
531,7,7.0,https://www.data.go.kr/data/15045878/fileData.do,None
532,6,6.0,https://www.data.go.kr/data/15045877/fileData.do,None
533,5,5.0,https://www.data.go.kr/data/15045876/fileData.do,None


In [204]:
df_re = df[['id', 'dataKey', 'category', 'categoryName', 'title', 'description',
       'startDate', 'endDate', 'viewCount', 'createAt', 'createByName',
       'owner', 'updateAt', 'totalFavCount', 'accessUrl', 'serviceType']]
df_re.rename(columns = {'id' : 'catalog_id'}, inplace = True)

df_re.to_csv('../result/제주데이터허브-catalog-2022-04-12.csv', index = False, encoding = 'utf-8')